# Trabajo Practico 2: Filtros y Deteccion de bordes
<p style='text-align: justify;'>
    En este trabajo practico se implementa un algoritmo que:

   * Calcula los gradientes en la imagen (direccion y modulo)
   * Muestra imagenes de angulo y modulo
   * Marca con color las direcciones de gradientes mas altos
    
    Para ello, haremos uso del algoritmo de Canny. (https://docs.opencv.org/3.4/da/d22/tutorial_py_canny.html)
</p>

In [18]:
# %matplotlib inline
%matplotlib

# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV 
import cv2 as cv
# Usamos las poderosas herramientas de graficación de matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt
# Importamos librerías para manejo de tiempo
import time

Using matplotlib backend: Qt5Agg


In [19]:
img = cv.imread('metalgrid.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

<p style='text-align: justify;'>
    En primer lugar utilizaremos un filtro gaussiano 5x5 con media 0 para reducir el ruido de la imagen.
</p>

In [20]:
# Suavizado Gaussiano
#====================
start = time.time()

blur = cv.GaussianBlur(gray,(5,5),0)

elapsed = time.time()-start
print('Blur {} segundos'.format(elapsed))

Blur 0.000997304916381836 segundos


<p style='text-align: justify;'>
    El siguiente paso es encontrar el gradiente de intensindad de la imagen. Utilizaremos un filtro de Sobel en direccion X e Y
</p>

In [23]:
# Gradientes
#===========
start = time.time()

# Aplicamos Sobelx en 'float32' y luego convertimos de nuevo a 8-bit para evitar overflow
sobelx_64 = cv.Sobel(blur,cv.CV_32F,1,0,ksize=3)
absx_64 = np.absolute(sobelx_64)
sobelx_8u1 = absx_64/absx_64.max()*255
sobelx_8u = np.uint8(sobelx_8u1)

# De igual modo para Sobely
sobely_64 = cv.Sobel(blur,cv.CV_32F,0,1,ksize=3)
absy_64 = np.absolute(sobely_64)
sobely_8u1 = absy_64/absy_64.max()*255
sobely_8u = np.uint8(sobely_8u1)

# De los gradiente calculamos Magnitud y lo pasamos a 8-bit (Opcional)
mag = np.hypot(sobelx_8u, sobely_8u)
mag = mag/mag.max()*255
mag = np.uint8(mag)

# Encontramos la Dirección y la pasamos a grados
theta = np.arctan2(sobely_64, sobelx_64)
angle = np.rad2deg(theta)

elapsed = time.time()-start
print('Gradientes {} segundos'.format(elapsed))

Gradientes 0.019918203353881836 segundos


In [58]:
# Supresión de no-máximos
#========================
start = time.time()

# Encontramos los píxels vecinos (b,c) en la dirección (redondeadea) del gradiente y aplicamos la supresión de no-máximos
M, N = mag.shape
Non_max = np.zeros((M,N), dtype= np.uint8)

horizontal = []
# Borde:


for i in range(1,M-1):
    for j in range(1,N-1):
        # Horizontal 0
        if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180) or (-22.5 <= angle[i,j] < 0) or (-180 <= angle[i,j] < -157.5):
            b = mag[i, j+1]
            c = mag[i, j-1]
            horizontal.append((i,j))
        # Diagonal 45
        elif (22.5 <= angle[i,j] < 67.5) or (-157.5 <= angle[i,j] < -112.5):
            b = mag[i+1, j+1]
            c = mag[i-1, j-1]
        # Vertical 90
        elif (67.5 <= angle[i,j] < 112.5) or (-112.5 <= angle[i,j] < -67.5):
            b = mag[i+1, j]
            c = mag[i-1, j]
        # Diagonal 135
        elif (112.5 <= angle[i,j] < 157.5) or (-67.5 <= angle[i,j] < -22.5):
            b = mag[i+1, j-1]
            c = mag[i-1, j+1]           
        
        # Supresión de no-máximos
        if (mag[i,j] >= b) and (mag[i,j] >= c):
            Non_max[i,j] = mag[i,j]
        else:
            Non_max[i,j] = 0

elapsed = time.time()-start
print('No-máximos {} segundos'.format(elapsed))

# Umbralizado por histéresis
#===========================
start = time.time()

UmbralAlto = 21
UmbralBajo = 15

M, N = Non_max.shape
out = np.zeros((M,N), dtype= np.uint8)

# Si la intensidad de un borde es mayor a 'UmbralAlto' es seguro un umbral, debajo de 'UmbralBajo' seguro no es
strong_i, strong_j = np.where(Non_max >= UmbralAlto)
zeros_i, zeros_j = np.where(Non_max < UmbralBajo)

# Bordes débiles
weak_i, weak_j = np.where((Non_max <= UmbralAlto) & (Non_max >= UmbralBajo))

# Setear el mismo valor a todos los tipos de píxel
out[strong_i, strong_j] = 255
out[zeros_i, zeros_j ] = 0
out[weak_i, weak_j] = 75

elapsed = time.time()-start
print('Histéresis {} segundos'.format(elapsed))

# Armo la imagen de bordes
#=========================
start = time.time()
M, N = out.shape
for i in range(1, M-1):
    for j in range(1, N-1):
        if (out[i,j] == 75):
            if 255 in [out[i+1, j-1],out[i+1, j],out[i+1, j+1],out[i, j-1],out[i, j+1],out[i-1, j-1],out[i-1, j],out[i-1, j+1]]:
                out[i, j] = 255
            else:
                out[i, j] = 0

elapsed = time.time()-start
print('Armado imagen bordes {} segundos'.format(elapsed))

print(horizontal)

#for h in range(len(horizontal)-1):
#    cv.line(out,horizontal[h],horizontal[h+1],(0,0,255),2,cv.LINE_AA)

# Muestro la imagen
#==================
cv.imshow("Canny",out)
plt.imshow(out)
plt.show()

No-máximos 3.5474448204040527 segundos
Histéresis 0.005999565124511719 segundos
Armado imagen bordes 0.3290698528289795 segundos
[(1, 3), (1, 13), (1, 14), (1, 22), (1, 24), (1, 62), (1, 63), (1, 64), (1, 68), (1, 69), (1, 76), (1, 92), (1, 93), (1, 99), (1, 103), (1, 115), (1, 119), (1, 123), (1, 125), (1, 126), (1, 143), (1, 145), (1, 154), (1, 155), (1, 162), (1, 163), (1, 164), (1, 173), (1, 174), (1, 175), (1, 176), (1, 199), (1, 203), (1, 209), (1, 214), (1, 223), (1, 224), (1, 225), (1, 226), (1, 262), (1, 263), (1, 264), (1, 268), (1, 269), (1, 276), (1, 292), (1, 293), (1, 299), (1, 303), (1, 315), (1, 319), (1, 323), (1, 325), (1, 326), (1, 343), (1, 345), (1, 354), (1, 355), (1, 362), (1, 363), (1, 364), (1, 373), (1, 374), (1, 375), (1, 376), (1, 399), (1, 403), (1, 409), (1, 414), (1, 423), (1, 424), (1, 425), (1, 426), (1, 462), (1, 463), (1, 464), (1, 468), (1, 469), (1, 476), (1, 492), (1, 493), (1, 499), (1, 503), (1, 515), (1, 519), (1, 523), (1, 525), (1, 526), (1, 5